## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
bin9_2 = pd.read_csv("../../바이너리_최종데이터_1004/gcc9/o2/o2_bincore9.csv", index_col=0)
print(bin9_2.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
bin9_2.reset_index(inplace=True, drop=True)

print('shape -', bin9_2.shape)
print('reset_index 완료')
print('input data shape')
bin9_2.head()

(33242498, 2)
shape - (33242498, 2)
reset_index 완료
input data shape


,bin,label
0,47,0
1,108,0
2,105,0
3,98,0
4,54,0


In [2]:
# (2-1) 데이터체크 1 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 
# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(bin9_2['bin'].unique()))

# (2-2) 데이터 체크 2 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(bin9_2['label'].value_counts())

256
0    33202108
1       40390
Name: label, dtype: int64


## (3) N Byte씩 자르기

In [3]:
idx_bin = bin9_2[bin9_2['label']==1].index  # 407, 474 ...
ls_bin = list(idx_bin)

# 최종 뽑을 행에 대한 index
ls_idx_bin = []

# n byte 자르기 방식
left_idx, right_idx = 0, 192 # 3개씩

# n byte 자르기
for k in range(left_idx, right_idx):
    ls_idx_bin.extend(list(idx_bin + k)) # index 형이라서 가능

#ls_idx = list(set(ls_idx)) 
ls_idx_bin.sort() # 인덱스 정렬

# 1차 index 해당범위 초과한 것들 없애기
ls_idx_bin = list(filter(lambda x: x<len(bin9_2), ls_idx_bin))
print(len(ls_idx_bin))

# 2차 남은 index들 중 right_idx 나눈 나머지 없애기
sub_bin = len(ls_idx_bin)%(right_idx)
print('나머지', sub_bin)

ls_idx_bin = ls_idx_bin[:len(ls_idx_bin)-sub_bin]
print('최종 길이', len(ls_idx_bin))

print('bin9_2', len(ls_idx_bin))

# loc 로 수정필요
bin9_2_Ngram = bin9_2.loc[ls_idx_bin,:].copy()

7754880
나머지 0
최종 길이 7754880
bin9_2 7754880


## (4) false data 만들기

In [4]:
# false data 만들기 - False 데이터 랜덤 생성

# 목표치
goal_bin = len(bin9_2_Ngram)/right_idx
count_bin = 0

print(goal_bin)

# 최종 데이터 Frame
d_bin = pd.DataFrame(columns = bin9_2.columns)

binutils_df = []
# goal 에 도달할 때까지
while True:
    if (count_bin == goal_bin):
            break
    # 진행상황 살펴보기 위함
            
    # 랜덤 N 바이트씩 뽑음
    # random index
    random_idx_bin = np.random.randint(len(bin9_2)-right_idx)

    if count_bin % 1000==0:
        print(count_bin, end=' ')
        print(random_idx_bin)

    df_bin = bin9_2[random_idx_bin : random_idx_bin + right_idx]
    
    # 뽑은 index의 N 바이트 중에 1이 없는 경우만
    if 1 not in df_bin['label'] and count_bin < goal_bin:
        binutils_df.append(df_bin)
        count_bin+=1

print('완료')
print(len(binutils_df))

40390.0
0 8422515
1000 26586934
2000 26668116
3000 6993481
4000 23691852
5000 20294314
6000 22170250
7000 26094728
8000 30949550
9000 11785211
10000 3664108
11000 25424576
12000 19599422
13000 27354370
14000 1727323
15000 23685894
16000 18654918
17000 13387227
18000 23614276
19000 28723468
20000 33161716
21000 30099717
22000 6101403
23000 19601592
24000 20878164
25000 21756814
26000 30097003
27000 19708857
28000 8240114
29000 9741938
30000 7441967
31000 4082534
32000 1854416
33000 9578642
34000 926333
35000 28115719
36000 27143442
37000 2383878
38000 20512187
39000 23377892
40000 23258917
완료
40390


In [5]:
# True data와 False Data 같은지 체크
print(len(binutils_df))
print(bin9_2['label'].value_counts()[1])

40390
40390


## (5) False Data + True Data 합치기

In [6]:
f_data = pd.concat(binutils_df)
final = pd.concat([f_data, bin9_2_Ngram])
final.shape

(15509760, 2)

## (6) one hot encoding

In [7]:
# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
bc9_2_onehot_Ngram = pd.get_dummies(final['bin'])
bc9_2_onehot_Ngram = pd.concat([final['label'], bc9_2_onehot_Ngram], axis=1)

print('원핫인코딩완료')
print(bc9_2_onehot_Ngram.shape)

원핫인코딩완료
(15509760, 257)


In [8]:
# 훈련 데이터, 훈련 라벨
x_bc9_2 = bc9_2_onehot_Ngram.iloc[:,1:].to_numpy()
y_bc9_2 = bc9_2_onehot_Ngram['label'].to_numpy()
print(x_bc9_2.shape, x_bc9_2.shape)

x_bc9_2 = x_bc9_2.reshape(-1, right_idx, x_bc9_2.shape[1])
y_bc9_2 = y_bc9_2.reshape(-1, right_idx, 1)

print(x_bc9_2.shape, y_bc9_2.shape)

(15509760, 256) (15509760, 256)
(80780, 192, 256) (80780, 192, 1)


In [9]:
# numpy 행, 열 섞기
p = np.random.permutation(x_bc9_2.shape[0])

x_bc9_2 = x_bc9_2[p]
y_bc9_2 = y_bc9_2[p]

print(x_bc9_2.shape, y_bc9_2.shape)

(80780, 192, 256) (80780, 192, 1)


In [10]:
np.save('x_bc9_2', x_bc9_2)
np.save('y_bc9_2', y_bc9_2)

## (7) 모델

In [11]:
# (10) 양방향 LSTM 모델링 작업
from tensorflow.keras import layers, models
#from tf.keras.models import Model, Sequential
#from tf.keras.layers import SimpleRNN, Input, Dense, LSTM
#from tf.keras.layers import Bidirectional, TimeDistributed

# 학습
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 10) # 조기종료 콜백함수 정의

xInput = layers.Input(batch_shape=(None,right_idx, 256)) 
xBiLstm = layers.Bidirectional(layers.LSTM(288, return_sequences=True, stateful=False), merge_mode = 'concat')(xInput)
xOutput = layers.TimeDistributed(layers.Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

## (8) 학습

In [12]:
model1 = models.Model(xInput, xOutput)
model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model1.summary()
import time
print('======Training stage======')
starttime = time.time()

model1.fit(x_bc9_2,
           y_bc9_2,
           epochs = 500,
           batch_size = 32,
           validation_split=0.3,
           callbacks=[early_stopping])

endtime = time.time()
print('걸린시간:', endtime - starttime)

model1.save('gcc9_bin_core_s192_h288_o2.h5')
print('save 완료')

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 192, 256)]        0         
_________________________________________________________________
bidirectional (Bidirectional (None, 192, 576)          1255680   
_________________________________________________________________
time_distributed (TimeDistri (None, 192, 1)            577       
Total params: 1,256,257
Trainable params: 1,256,257
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 56546 samples, validate on 24234 samples
Epoch 1/500
56546/56546 [==============================] - 79s 1ms/sample - loss: 0.0124 - accuracy: 0.9962 - val_loss: 0.0049 - val_accuracy: 0.9984
Epoch 2/500
56546/56546 [==============================] - 72s 1ms/sample - loss: 0.0039 - accuracy: 0.9987 - val_loss: 0.0060 - val_accuracy: 0.9980
Epoc

걸린시간: 1314.158191204071
save 완료
